In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/comment-label/__results__.html
/kaggle/input/comment-label/comments_all_labeled.csv
/kaggle/input/comment-label/__notebook__.ipynb
/kaggle/input/comment-label/__output__.json
/kaggle/input/comment-label/custom.css
/kaggle/input/reply-label-limit/__results__.html
/kaggle/input/reply-label-limit/reply_label.csv
/kaggle/input/reply-label-limit/__notebook__.ipynb
/kaggle/input/reply-label-limit/__output__.json
/kaggle/input/reply-label-limit/custom.css


In [2]:
import polars as pl
from tqdm import tqdm
import os
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy
import time

In [3]:
comment = pd.read_csv('/kaggle/input/comment-label/comments_all_labeled.csv')
comment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3560004 entries, 0 to 3560003
Data columns (total 6 columns):
 #   Column           Dtype  
---  ------           -----  
 0   comment_id       object 
 1   user_id          object 
 2   course_id        object 
 3   text             object 
 4   create_time      object 
 5   sentiment_label  float64
dtypes: float64(1), object(5)
memory usage: 163.0+ MB


In [4]:
reply = pd.read_csv('/kaggle/input/reply-label-limit/reply_label.csv')
reply.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46969 entries, 0 to 46968
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   reply_id         46969 non-null  object 
 1   comment_id       46969 non-null  object 
 2   user_id          46969 non-null  object 
 3   course_id        46969 non-null  object 
 4   text             46969 non-null  object 
 5   create_time      46969 non-null  object 
 6   sentiment_label  46968 non-null  float64
dtypes: float64(1), object(6)
memory usage: 2.5+ MB


# Phân loại cảm xúc của comment

In [5]:
# Bỏ đi những cột không cần thiết
comment.drop(columns=["text", "comment_id", "create_time"], inplace=True)
comment.head()

,user_id,course_id,sentiment_label
0,U_5518487,C_735269,1.0
1,U_5518487,C_735269,1.0
2,U_11459568,C_682656,0.0
3,U_11423362,C_682656,0.0
4,U_11370758,C_682656,1.0


In [6]:
# Đếm số comment positive, negative, neutral và tổng số comment
comment['positive'] = comment['sentiment_label'].apply(lambda x: 1 if x == 2.0 else 0)
comment['negative'] = comment['sentiment_label'].apply(lambda x: 1 if x == 0.0 else 0)
comment['neutral'] = comment['sentiment_label'].apply(lambda x: 1 if x == 1.0 else 0)

comment.drop(columns=["sentiment_label"], inplace=True)
comment = comment.groupby(["course_id", "user_id"]).sum().reset_index()
comment['total_cmt'] = comment['positive'] + comment['negative'] + comment['neutral']

comment.head()

,course_id,user_id,positive,negative,neutral,total_cmt
0,C_1017355,U_1003583,0,0,9,9
1,C_1017355,U_10395255,0,0,1,1
2,C_1017355,U_10448165,0,0,8,8
3,C_1017355,U_10448173,1,0,7,8
4,C_1017355,U_10448177,0,0,1,1


# Phân loại cảm xúc của reply

In [7]:
# Bỏ đi những cột không cần thiết
reply.drop(columns=["text", "comment_id", "create_time", "reply_id"], inplace=True)
reply.head()

,user_id,course_id,sentiment_label
0,U_11370564,C_682656,1.0
1,U_11563268,C_682656,0.0
2,U_11389118,C_682656,0.0
3,U_11388224,C_682656,1.0
4,U_2005701,C_676705,1.0


In [8]:
# Đếm số reply positive, negative, neutral và tổng số reply
reply['positive'] = reply['sentiment_label'].apply(lambda x: 1 if x == 2.0 else 0)
reply['negative'] = reply['sentiment_label'].apply(lambda x: 1 if x == 0.0 else 0)
reply['neutral'] = reply['sentiment_label'].apply(lambda x: 1 if x == 1.0 else 0)

reply.drop(columns=["sentiment_label"], inplace=True)
reply = reply.groupby(["course_id", "user_id"]).sum().reset_index()
reply['total_reply'] = reply['positive'] + reply['negative'] + reply['neutral']

reply.head()

,course_id,user_id,positive,negative,neutral,total_reply
0,C_1017355,U_10539786,1,0,0,1
1,C_1017355,U_12595791,0,0,1,1
2,C_1017355,U_13575454,0,0,1,1
3,C_1017355,U_14277532,8,2,12,22
4,C_1017355,U_15163944,0,0,1,1


# Tổng hợp

In [9]:
final_df = pd.merge(comment, reply, on=['user_id', 'course_id'], how='outer', suffixes=('_comment', '_reply'))
final_df.fillna(0, inplace=True)
final_df.head()

,course_id,user_id,positive_comment,negative_comment,neutral_comment,total_cmt,positive_reply,negative_reply,neutral_reply,total_reply
0,C_1779421,U_1001314,0.0,1.0,4.0,5.0,0.0,0.0,0.0,0.0
1,C_735164,U_1001563,1.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0
2,C_735164,U_1001625,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
3,C_735164,U_1001703,2.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0
4,C_1903976,U_1001806,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


In [10]:
# Cộng các giá trị trong các cột 'positive', 'negative', 'neutral' từ cả comment và reply
final_df['positive'] = final_df['positive_comment'] + final_df['positive_reply']
final_df['negative'] = final_df['negative_comment'] + final_df['negative_reply']
final_df['neutral'] = final_df['neutral_comment'] + final_df['neutral_reply']

# Tính tổng số comment và reply
final_df['total'] = final_df['total_reply'] + final_df['total_cmt']

final_df.drop(columns=['positive_comment', 'negative_comment', 'neutral_comment', 'total_reply', 'positive_reply', 'negative_reply', 'neutral_reply', 'total_cmt'], inplace=True)
final_df.head()

,course_id,user_id,positive,negative,neutral,total
0,C_1779421,U_1001314,0.0,1.0,4.0,5.0
1,C_735164,U_1001563,1.0,0.0,1.0,2.0
2,C_735164,U_1001625,0.0,0.0,1.0,1.0
3,C_735164,U_1001703,2.0,0.0,2.0,4.0
4,C_1903976,U_1001806,0.0,0.0,1.0,1.0
